In [1]:
!pip install pyserial#installs package

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 1.9 MB/s eta 0:00:00.5 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [18]:
import serial
import time
import csv
from datetime import datetime
import serial.tools.list_ports

# List available COM ports
print("Available COM ports:", [p.device for p in serial.tools.list_ports.comports()])



Available COM ports: []


In [ ]:
port = 'COM9'  # Adjust as necessary
baud = 9600

try:#prevents crashes ,makes code more robust
    ser = serial.Serial(port, baud, timeout=1)#object
    time.sleep(2)  # Wait for Arduino to initialize

    filename = "dht11_log.csv"#creatrs csv for analysis
    with open(filename, "w", newline='') as f: #opens the file
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Humidity", "Temperature"])#creates headers

        for _ in range(50):#for loop,collects 50 readings
            try:
                line = ser.readline().decode('utf-8', errors='ignore').strip()#reads data from arduino
                print(f"Raw line: {line}")

                if "Humidity" in line and "Temperature" in line:#string manipulation to isolate readings
                    h = float(line.split("Humidity: ")[1].split(" %")[0])
                    t = float(line.split("Temperature: ")[1].split(" °C")[0])
                    ts = datetime.now().isoformat()
                    writer.writerow([ts, h, t])
                    print(f"{ts} - {h}% - {t}°C")

            except Exception as e:
                print(f"Error processing line: {e}")
                continue

    ser.close()

except serial.SerialException as e:
    print(f"Serial port error: {e}")